In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
file = 'C:\\Users\\daun5\\jupyter_notebook\\EC-Advance_daun\\room_list_Gongneung-dong.csv'
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
room_data = pd.read_csv(file, encoding='cp949')

In [4]:
def split_price(x): #price_info 전처리
    x = x.replace('[', ' ').replace("'", ' ').replace(']', ' ')
    x = x.split(',')
    
    return x
def split_options(x): #options 전처리
    x = x.replace("'", ' ').replace("'", ' ').replace

In [5]:
i = 0; #관리비 전처리
for data in room_data['maintenance_cost_str']:
    if data == '없음':
        room_data['maintenance_cost_str'][i] = 0
    else:
        room_data['maintenance_cost_str'][i] = float(data[:-3])
    i = i + 1;

<ipython-input-5-ea54dfc5b833>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['maintenance_cost_str'][i] = 0
<ipython-input-5-ea54dfc5b833>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['maintenance_cost_str'][i] = float(data[:-3])


In [6]:
room_data['monthly_rent'] = room_data['price_info_str'].apply(lambda x:split_price(x)).str[1]
room_data['deposit'] = room_data['price_info_str'].apply(lambda x:split_price(x)).str[0]
#price_info 를 월세와 보증금으로 분리

In [7]:
room_data.insert(6, 'monthly_total', 0.0)
#월세+관리비 총합으로 monthly_total 열 만듦

In [8]:
room_data['maintenance_cost_str'] = pd.to_numeric(room_data['maintenance_cost_str'])
room_data['monthly_rent'] = pd.to_numeric(room_data['monthly_rent'])
i = 0;
for j in room_data['deposit']:
    if('억' in j):
        room_data['deposit'][i] = room_data['deposit'][i].replace('억', ' ')
        room_data['deposit'][i] = room_data['deposit'][i].replace(' ', '')
    i += 1
room_data['deposit'] = pd.to_numeric(room_data['deposit'])
#관리비, 월세, 보증금 문자형을 숫자형으로 변환

In [9]:
i = 0;
for j in room_data['room_id']:
    room_data['monthly_total'][i] = room_data['monthly_rent'][i] + room_data['maintenance_cost_str'][i]
    i += 1
#보증금+월세해서 monthly_total에 대입

<ipython-input-9-2c63176b6ed5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['monthly_total'][i] = room_data['monthly_rent'][i] + room_data['maintenance_cost_str'][i]


In [10]:
j = 0 #층수 전처리, 중층과 옥탑은 삭제, 반지층은 0, 1~4층은 1, 5층 이상은 2
for i in room_data['room_floor_str']: 
    room_data['room_floor_str'][j] = i.replace('층', ' ')
    if (i == '중층' or i == '옥탑'):
        room_data = room_data.drop(j, axis = 0)
        room_data = room_data.reset_index(drop = True)
        j -= 1
    elif (i == '반지층'):
        room_data['room_floor_str'][j] = 0
    j += 1

room_data['room_floor_str'] = pd.to_numeric(room_data['room_floor_str'])

<ipython-input-10-26a76ecfa878>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['room_floor_str'][j] = i.replace('층', ' ')
<ipython-input-10-26a76ecfa878>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['room_floor_str'][j] = 0


In [11]:
j = 0
for i in room_data['room_floor_str']:
    if (5 > i > 0):
        room_data['room_floor_str'][j] = 1
    elif (i >= 5):
        room_data['room_floor_str'][j] = 2
    j += 1

<ipython-input-11-16081f7da9fd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['room_floor_str'][j] = 1
<ipython-input-11-16081f7da9fd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['room_floor_str'][j] = 2


In [12]:
rule_gasrange = [room_data['room_options'].str.contains('가스레인지')]
rule_frige = [room_data['room_options'].str.contains('냉장고')]
rule_wash = [room_data['room_options'].str.contains('세탁기')]
rule_shoes = [room_data['room_options'].str.contains('신발장')]
rule_doorlock = [room_data['room_options'].str.contains('전자도어락')]
rule_air = [room_data['room_options'].str.contains('에어컨')]
rule_induction = [room_data['room_options'].str.contains('인덕션')]
rule_closet = [room_data['room_options'].str.contains('옷장')]
rule_microwave = [room_data['room_options'].str.contains('전자레인지')]
rule_desk = [room_data['room_options'].str.contains('책상')]
rule_bed = [room_data['room_options'].str.contains('침대')]

rulell = [ 1 ]

room_data['gasrange'] = np.select(rule_gasrange, rulell)
room_data['frige'] = np.select(rule_frige, rulell)
room_data['wash'] = np.select(rule_wash, rulell)
room_data['shoes'] = np.select(rule_shoes, rulell)
room_data['doorlock'] = np.select(rule_doorlock, rulell)
room_data['air'] = np.select(rule_air, rulell)
room_data['induction'] = np.select(rule_induction, rulell)
room_data['closet'] = np.select(rule_closet, rulell)
room_data['microwave'] = np.select(rule_microwave, rulell)
room_data['desk'] = np.select(rule_desk, rulell)
room_data['bed'] = np.select(rule_bed, rulell)
#옵션마다 열 만들어서 있으면 1, 없으면 0로 처리 
#contains 문자열 있으면 True, 없으면 False로 반환하는 함수
#select 

In [13]:
j = 0
for i in room_data['room_id']:
    room_data['direction_str'][j] = np.nan_to_num(room_data['direction_str'][j])
    j += 1
j = 0
for i in room_data['direction_str']:
    if (i == 0):
        room_data['direction_str'][j] = '없음'
    j += 1

<ipython-input-13-ed8e2b8a5770>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['direction_str'][j] = np.nan_to_num(room_data['direction_str'][j])
<ipython-input-13-ed8e2b8a5770>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['direction_str'][j] = '없음'


In [14]:
rule_zero = [room_data['direction_str'].str.contains('없음')]
rule_north_east = [room_data['direction_str'].str.contains('북동')]
rule_north_west = [room_data['direction_str'].str.contains('북서')]
rule_south_east = [room_data['direction_str'].str.contains('남동')]
rule_south_west = [room_data['direction_str'].str.contains('남서')]

room_data['direction_str'] = room_data['direction_str'].replace('북동', ' ')
room_data['direction_str'] = room_data['direction_str'].replace('북서', ' ')
room_data['direction_str'] = room_data['direction_str'].replace('남동', ' ')
room_data['direction_str'] = room_data['direction_str'].replace('남서', ' ')
    
rule_north= [room_data['direction_str'].str.contains('북')]
rule_east= [room_data['direction_str'].str.contains('동')]
rule_south = [room_data['direction_str'].str.contains('남')]
rule_west= [room_data['direction_str'].str.contains('서')]

In [15]:
room_data['no_direction'] = np.select(rule_zero, rulell)
room_data['north_west'] = np.select(rule_north_west, rulell)
room_data['north'] = np.select(rule_north, rulell)
room_data['north_east'] = np.select(rule_north_east, rulell)
room_data['east'] = np.select(rule_east, rulell)
room_data['south_east'] = np.select(rule_south_east, rulell)
room_data['south'] = np.select(rule_south, rulell)
room_data['south_west'] = np.select(rule_south_west, rulell)
room_data['west'] = np.select(rule_west, rulell)

In [16]:
j = 0
for i in room_data['building_approval_date_str']:
    if (i != '-'):
        room_data['building_approval_date_str'][j] = pd.to_datetime(room_data['building_approval_date_str'][j])
    else:
        room_data = room_data.drop(j, axis = 0)
        room_data = room_data.reset_index(drop = True)
        j -= 1
    j += 1

<ipython-input-16-fb0a2bf869e7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['building_approval_date_str'][j] = pd.to_datetime(room_data['building_approval_date_str'][j])


In [17]:
year_now = datetime.today().year
month_now = datetime.today().month
month_total_now = year_now * 12 + month_now
room_data.insert(4, 'age_month', 0)

In [18]:
j = 0
for i in room_data['building_approval_date_str']:
        year_data = room_data['building_approval_date_str'][j].year
        month_data = room_data['building_approval_date_str'][j].month
        month_total_data = year_data * 12 + month_data
        room_data['age_month'][j] = month_total_now - month_total_data
        j += 1

<ipython-input-18-7b0421ad63ec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['age_month'][j] = month_total_now - month_total_data


In [19]:
j = 0
for i in room_data['safeties']:
    room_data['safeties'][j] = np.nan_to_num(room_data['safeties'][j])
    j += 1

<ipython-input-19-f6f2877e882c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['safeties'][j] = np.nan_to_num(room_data['safeties'][j])


In [20]:
j = 0
for i in room_data['safeties']:
    if (i == 0):
        room_data['safeties'][j] = '없음'
    j += 1

<ipython-input-20-08af8d5dbd4a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_data['safeties'][j] = '없음'


In [21]:
rule_videophone = [room_data['safeties'].str.contains('비디오폰')]
rule_robby = [room_data['safeties'].str.contains('공동현관')]
rule_man = [room_data['safeties'].str.contains('경비원')]
rule_cctv = [room_data['safeties'].str.contains('CCTV')]
rule_window = [room_data['safeties'].str.contains('방범창')]
rule_intercom = [room_data['safeties'].str.contains('인터폰')]
rule_cardkey = [room_data['safeties'].str.contains('카드키')]
rule_firealarm = [room_data['safeties'].str.contains('화재경보기')]

room_data['videophone'] = np.select(rule_videophone, rulell)
room_data['robby'] = np.select(rule_robby, rulell)
room_data['man'] = np.select(rule_man, rulell)
room_data['cctv'] = np.select(rule_cctv, rulell)
room_data['window'] = np.select(rule_window, rulell)
room_data['intercom'] = np.select(rule_intercom, rulell)
room_data['cardkey'] = np.select(rule_cardkey, rulell)
room_data['firealarm'] = np.select(rule_firealarm, rulell)

In [22]:
room_data = room_data.drop(columns=['price_info_str', 
                                    'maintenance_cost_str', 
                                    'monthly_rent', 
                                    'room_options', 
                                    'direction_str', 
                                    'no_direction', 
                                    'building_approval_date_str',
                                    'safeties'], axis=1)

In [23]:
with pd.ExcelWriter("./data.xlsx") as writer:
    room_data.to_excel(writer,sheet_name="시트1",index=False) 